# 有馬記念に勝ちたい
ここら辺を参考に予測器を作って有馬記念に勝ちたい

- https://qiita.com/yossymura/items/334a8f3ef85bff081913
- https://qiita.com/ishizakiiii/items/3b894b6e987fdf87093e
- https://qiita.com/kami634/items/55e49dad76396d808bf5
- https://qiita.com/teco_ito/items/171865d1d6227a5e85e8

中山競馬場での予測器を作る

## データの収集
中山競馬場の各レースについて、出走馬の当日のデータ、過去のレースのデータをhttps://db.netkeiba.com/　からとってくる

[レースごとのページ](https://db.netkeiba.com/race/201908050311/)、[馬ごとのページ](https://db.netkeiba.com/horse/2017105327/)があって、馬ごとのページのみ見れば十分なので、中山競馬場のレースに出走した馬のurlを収集してそこからデータを作る

しかし、中山競馬場に出走した馬という検索ができないので、中山競馬場のレース->各レースページ->レースに出走した馬のページ  とアクセスする

In [71]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import sqlite3
from contextlib import closing
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm_notebook as tqdm
import time
import re
import datetime
from bs4 import BeautifulSoup
import requests

options = Options()
options.add_argument('--headless')    # ヘッドレスモードに
driver = webdriver.Chrome(chrome_options=options) 
wait = WebDriverWait(driver,10)

/Users/ht/.pyenv/versions/miniconda3-latest/envs/cert/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


In [41]:
URL = "https://db.netkeiba.com/?pid=race_search_detail"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

### 中山競馬場のレースのurlを収集

In [3]:
# 中山を選択
driver.find_element_by_id("check_Jyo_06").click()
# 新馬、未勝利、未出走以外の芝、ダートだけ選択
driver.find_element_by_id("check_track_1").click()
driver.find_element_by_id("check_track_2").click()
driver.find_element_by_id("check_grade_1").click()
driver.find_element_by_id("check_grade_2").click()
driver.find_element_by_id("check_grade_3").click()
driver.find_element_by_id("check_grade_4").click()
driver.find_element_by_id("check_grade_5").click()
driver.find_element_by_id("check_grade_6").click()
driver.find_element_by_id("check_grade_7").click()

# 表示件数を選択(20,50,100の中から最大の100へ)
list_element = driver.find_element_by_name('list')
list_select = Select(list_element)
list_select.select_by_value("100")
# フォームを送信
frm = driver.find_element_by_css_selector("#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

In [4]:
# url を収集
df = pd.DataFrame(columns='race_url year month date'.split(' '))

while True:
    time.sleep(0.5)
    wait.until(EC.presence_of_all_elements_located)
    all_rows = driver.find_element_by_class_name('race_table_01').find_elements_by_tag_name("tr")
    for row in range(1, len(all_rows)):
        tds = all_rows[row].find_elements_by_tag_name("td")
        year = tds[0].find_element_by_tag_name("a").text
        race_href = tds[4].find_element_by_tag_name("a").get_attribute("href")
        tmp_se = pd.Series([race_href]+[int(i) for i in year.split('/')], index=df.columns)
        df = df.append(tmp_se, ignore_index=True)
    try:
        target = driver.find_elements_by_link_text("次")[0]
        driver.execute_script("arguments[0].click();", target) #javascriptでクリック処理
    except IndexError:
        break
df.to_csv('./data/races.csv')
df

,race_url,year,month,date
0,https://db.netkeiba.com/race/201906050607/,2019,12,15
1,https://db.netkeiba.com/race/201906050608/,2019,12,15
2,https://db.netkeiba.com/race/201906050609/,2019,12,15
3,https://db.netkeiba.com/race/201906050610/,2019,12,15
4,https://db.netkeiba.com/race/201906050611/,2019,12,15
...,...,...,...,...
8130,https://db.netkeiba.com/race/195506040810/,1955,10,16
8131,https://db.netkeiba.com/race/195506040410/,1955,10,2
8132,https://db.netkeiba.com/race/195506010401/,1955,1,8
8133,https://db.netkeiba.com/race/195406060410/,1954,12,12


In [5]:
df[df['year']>=2000]

,race_url,year,month,date
0,https://db.netkeiba.com/race/201906050607/,2019,12,15
1,https://db.netkeiba.com/race/201906050608/,2019,12,15
2,https://db.netkeiba.com/race/201906050609/,2019,12,15
3,https://db.netkeiba.com/race/201906050610/,2019,12,15
4,https://db.netkeiba.com/race/201906050611/,2019,12,15
...,...,...,...,...
5039,https://db.netkeiba.com/race/200006010211/,2000,1,6
5040,https://db.netkeiba.com/race/200006010107/,2000,1,5
5041,https://db.netkeiba.com/race/200006010108/,2000,1,5
5042,https://db.netkeiba.com/race/200006010109/,2000,1,5


### レースに出走した馬のurlを収集

途中で気づいたけどBeautiful Soup でやった方が速かった

In [ ]:
# urls = []
for i, url in enumerate(tqdm(df['race_url'])):
    # 一回止まったので途中から再開
    if i < 5619:
        continue
    driver.get(url)
    time.sleep(1)
    wait.until(EC.presence_of_all_elements_located)
    try:
        all_rows = driver.find_element_by_xpath('//table[@class="race_table_01 nk_tb_common"]').find_elements_by_tag_name("tr")
    except NoSuchElementException:
        print(url)
        continue
    for row in range(1, len(all_rows)):
        tds = all_rows[row].find_elements_by_tag_name("td")
        url_href = tds[3].find_element_by_tag_name("a").get_attribute("href")
        urls.append(url_href)
urls
with open('./data/horse_urls.pkl', 'wb') as f:
    pickle.dump(list(set(urls)), f)

### (中山競馬場のレースに出走した)馬の出走履歴の表をデータベースに突っ込む

#### 表

|idx|attr|例|
|-----|-----|-----|
|0|日付|2019/12/15|
|1|会場r|5中山6|
|2|天気|晴
|3|R|7
|4|レース名(href=race id)|href|
|5|映像(null)||
|6|頭数|14|
|7|枠番|5|
|8|馬番|7|
|9|odds|9.1|
|10|人気|4|
|11|着順|1|
|12|騎手|武豊|
|13|斤量|57|
|14|距離|ダ2500|
|15|馬場|良|
|16|馬場指数(null)||
|17|タイム|2:42.8|
|18|着差|2.1|
|19|タイム指数(null)||
|20|通過(2-2, 2-3-4)||
|21|ペース(float-float)|31.0-38.6|
|22|上り|38.0|
|23|馬体重(実体重(+-体重))|538(-12)|
|24~|使わん||

In [84]:
import re
import datetime

weight_ptn = re.compile(r'([0-9]+)\(([+-]?[0-9]+)\)')
place_ptn = re.compile(r'[0-9]*([^0-9]+)[0-9]*')
last_nums_ptn = re.compile(r'.*/([0-9]+)/')

data_index = 'horse_id R head_count wakuban umaban popularity order burden_weight odds time_gap up weather baba raceid place pace0 pace1 race_type distance weight weight_change date time'.split(' ')

def trynone(func, inp):
    try:
        return func(inp)
    except:
        return None
    
def fixed_length(l, n):
    return l + [None for _ in range(n-len(l))]

def tds2data(tds, horse_id, timer=None):
    intdata = [trynone(int, td.text) for td in np.array(tds)[[3, 6, 7, 8, 10, 11, 13]]]
    floatdata = [trynone(float, td.text) for td in np.array(tds)[[9, 18, 22]]]
    textdata = [td.text for td in np.array(tds)[[2, 15]]]
    raceid = [trynone(int, re.findall(last_nums_ptn, tds[4].find("a").get("href"))[0])]
    place = fixed_length(re.findall(place_ptn, tds[1].text), 1)
    pace = fixed_length([trynone(float, i) for i in tds[21].text.split('-')], 2)
    type_n_dist = [tds[14].text[0], trynone(int, tds[14].text[1:])]
    weight = fixed_length([trynone(int, i) for i in re.findall(weight_ptn, tds[23].text)[0]], 2)
    date = [datetime.datetime.strptime(tds[0].text, '%Y/%m/%d')]
    time = [datetime.datetime.strptime(tds[17].text, '%M:%S.%U')]
    data = [trynone(int, re.findall(last_nums_ptn, horse_id)[0])] + intdata + floatdata + textdata + raceid + place + pace + type_n_dist + weight + date + time
    return tuple(data)

In [ ]:
import sqlite3
from contextlib import closing
from tqdm import tqdm_notebook as tqdm
import time

class Timer:
    def __init__(self):
        t = time.time()
        self.start = t
        self.rap_start = t
    
    def time(self, description=None):
        t = time.time()
        print(f'{description}\nabs_time: {t-self.start}\nrap_time: {t-self.rap_start}')
        self.rap_start = t

dbpath = './data/horse.db'
if True:
    with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
        c = conn.cursor()
        c.execute('drop table if exists race')
        c.execute('''CREATE TABLE IF NOT EXISTS race
                     (horse_id int, R int, head_count int, wakuban int, umaban int, popularity int, race_order int, 
                      burden_weight int, odds float, time_gap float, up float, weather text, baba text,
                      raceid int, place text, pace0 float, pace1 float, race_type text, distance int, 
                      weight int, weight_change int, date datetime, time datetime)''')
        conn.commit()
if True:
    insert_sql = 'insert into race values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
    comm_datas = []
    with closing(sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)) as conn:
        c = conn.cursor()
        timer = Timer()
        with open('./error_datalog.txt', 'w') as errorlog:
            for i, url in enumerate(tqdm(sorted(list(set(urls)))[::-1])):
                if i <= 3602:
                    continue

                res = requests.get(url)
                time.sleep(0.2)
                soup = BeautifulSoup(res.content, 'html.parser')
                try:
                    all_rows = soup.find('table', class_='db_h_race_results nk_tb_common').find_all('tr')
                except AttributeError:
                    errorlog.write(str((url, i)))
                for row, tr in enumerate(all_rows):
                    if row == 0:
                        continue
                    tds = tr.find_all("td")
                    try:
                        data = tds2data(tds, url)
                    except (ValueError, IndexError, TypeError, AttributeError, KeyError) as e:
                        errorlog.write(str((url, i, row)))
                    if len(data) != 23:
                        continue
                    comm_datas.append(data)
                if i % 50 == 2:
                    c.executemany(insert_sql, comm_datas)
                    conn.commit()
                    comm_datas = []
                    timer.time(f'end {i}')
        c.executemany(insert_sql, comm_datas)
        conn.commit()
        timer.time('end')

/Users/ht/.pyenv/versions/miniconda3-latest/envs/cert/lib/python3.7/site-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


end 3652
abs_time: 88.578684091568
rap_time: 88.578684091568


## データセットの作成
データベースから各レースごとに最大24馬の過去五レースをとってくる
ない分は0埋め

## モデル作成
(A)過去5レース + そのレースの情報　からスコアづけするような部分 + (B)出走馬数ぶんのスコアから1位予想する部分　+ (C)1位確率とoddsから良い買い方を選択する部分

((A)*24)+(B)でまず学習、その後
(C)を学習

考え中

- とりあえず全部全結合で、Bは一層? Cも1層？
- A, B は予測にodds は用いない
- Aはlstmと合わせる?

### A
入力: 

- 当日情報
    - 数値データ
        - 前のレースから何日
        - 頭数
        - 枠版
        - 馬番
        - 斤量
        - 重量
        - 重量変化
        - 距離
        - R 
    - カテゴリ(one-hot でok)
        - レース種類
        - 天気
        - 馬場
- 過去五レース(当日情報で扱うデータに加え)
    - 数値データ
        - 着順
        - odds
        - タイム
        - 着差
        - ペース
        - 上り
    - カテゴリ(one-hot でok)
    
出力: score(float)

### B
入力: score*24

出力: 1位確率*24

loss = cross entoropy AとBで一回学習させる

### C
入力: 1位確率*24, odds*24

出力: 買うべき金額

loss = 金額の増減 でcのみ学習？
